# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [8]:
from sklearn.datasets import  fetch_california_housing
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error

from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

In [4]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [5]:
df = spaceship.dropna()
df['Cabin'] = df['Cabin'].str[0]
df = df.drop(['PassengerId','Name'], axis=1)
df_with_dummies = pd.get_dummies(df, columns=df.select_dtypes(include='object').columns)
target = df['Transported']
features = df_with_dummies.drop(['Transported'], axis=1)
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.20,random_state=42)

#an instance of the normalizer

normalizer = MinMaxScaler()
normalizer.fit(x_train)

x_train_norm = normalizer.transform(x_train)
x_test_norm = normalizer.transform(x_test)

x_train_norm = pd.DataFrame(x_train_norm, columns=features.columns)
x_test_norm = pd.DataFrame(x_test_norm, columns=features.columns)


/var/folders/9x/t81_t1w14_s8w2jy_nx_xn4m0000gn/T/ipykernel_45311/2342648828.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Cabin'] = df['Cabin'].str[0]


- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [10]:
baggin_cls = BaggingClassifier(DecisionTreeClassifier(max_depth = 8),
                             n_estimators=200,
                             max_samples=3000,
                             bootstrap=True)
baggin_cls.fit(x_train_norm,y_train)

,estimator,DecisionTreeC...r(max_depth=8)
,n_estimators,200
,max_samples,3000
,max_features,1.0
,bootstrap,True
,bootstrap_features,False
,oob_score,False
,warm_start,False
,n_jobs,None
,random_state,None
,verbose,0


- Evaluate your model

In [11]:
#evaluate model perfomance
pred = baggin_cls.predict(x_test_norm)


print("TEST DATA")

print("R2 score ", r2_score(y_test, pred))
print("RMSE ", root_mean_squared_error(y_test, pred))
print("MAE ", mean_absolute_error(y_test, pred))


print("----------------------------")

print("TRAIN DATA")
pred = baggin_cls.predict(x_train_norm)

print("R2 score ", r2_score(y_train, pred))
print("RMSE ", root_mean_squared_error(y_train, pred))
print("MAE ", mean_absolute_error(y_train, pred))

TEST DATA
R2 score  0.24043336835623552
RMSE  0.4357336831857647
MAE  0.18986384266263237
----------------------------
TRAIN DATA
R2 score  0.3292713886773293
RMSE  0.4094826040603057
MAE  0.16767600302800909


**Grid/Random Search**

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.

In [12]:
gradient_reg = GradientBoostingRegressor()
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.1, 0.01, 0.001],
    'max_depth': [3, 4, 5],
    'min_samples_split': [2, 4, 6],
    'min_samples_leaf': [1, 2, 3],
    'max_features': ['sqrt', 'log2', None]
}

- Run Grid Search

In [13]:
model = GridSearchCV(estimator = gradient_reg,
                     param_grid = param_grid,
                     cv = 5,
                     verbose = 10,
                     n_jobs=-1)
model.fit(x_train_norm, y_train)

Fitting 5 folds for each of 729 candidates, totalling 3645 fits
[CV 3/5; 1/729] START learning_rate=0.1, max_depth=3, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100
[CV 4/5; 1/729] START learning_rate=0.1, max_depth=3, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100
[CV 5/5; 1/729] START learning_rate=0.1, max_depth=3, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100
[CV 2/5; 2/729] START learning_rate=0.1, max_depth=3, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200
[CV 1/5; 1/729] START learning_rate=0.1, max_depth=3, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100
[CV 2/5; 1/729] START learning_rate=0.1, max_depth=3, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100
[CV 1/5; 2/729] START learning_rate=0.1, max_depth=3, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200
[CV 5/5; 1/729] END

,estimator,GradientBoostingRegressor()
,param_grid,"{'learning_rate': [0.1, 0.01, ...], 'max_depth': [3, 4, ...], 'max_features': ['sqrt', 'log2', ...], 'min_samples_leaf': [1, 2, ...], ...}"
,scoring,None
,n_jobs,-1
,refit,True
,cv,5
,verbose,10
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,loss,'squared_error'


- Evaluate your model

In [14]:
model.best_params_

{'learning_rate': 0.1,
 'max_depth': 5,
 'max_features': 'log2',
 'min_samples_leaf': 2,
 'min_samples_split': 6,
 'n_estimators': 100}

In [15]:
best_model = model.best_estimator_

In [16]:
best_model

,loss,'squared_error'
,learning_rate,0.1
,n_estimators,100
,subsample,1.0
,criterion,'friedman_mse'
,min_samples_split,6
,min_samples_leaf,2
,min_weight_fraction_leaf,0.0
,max_depth,5
,min_impurity_decrease,0.0
,init,None


In [17]:
pred = best_model.predict(x_test_norm)


print("r2 score: ", r2_score(y_test, pred))
print("RMSE: ", root_mean_squared_error(y_test, pred))
print("MAE: ", mean_absolute_error(y_test, pred))

r2 score:  0.47118237601161184
RMSE:  0.36357260726382584
MAE:  0.2724430625567576


In [18]:
pred = best_model.predict(x_train_norm)


print("r2 score: ", r2_score(y_train, pred))
print("RMSE: ", root_mean_squared_error(y_train, pred))
print("MAE: ", mean_absolute_error(y_train, pred))

r2 score:  0.5489326475092309
RMSE:  0.33580158182884323
MAE:  0.25298289066905977
